In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import notebook
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from tqdm import tqdm
from hloc.utils.read_write_model import read_images_binary,read_points3D_binary,read_cameras_binary,qvec2rotmat
from PIL import Image
from scipy.interpolate import griddata
import cv2
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from mpl_toolkits.mplot3d import Axes3D

In [3]:
images_path='/home/cvnar/Spatial_Contrastive_Loss/outputs/aachen/sfm_superpoint+superglue/images.bin'
points3Ds_path='/home/cvnar/Spatial_Contrastive_Loss/outputs/aachen/sfm_superpoint+superglue/points3D.bin'
caperas_path='/home/cvnar/Spatial_Contrastive_Loss/outputs/aachen/sfm_superpoint+superglue/cameras.bin'
images = read_images_binary(images_path)
points3Ds = read_points3D_binary(points3Ds_path)
cameras = read_cameras_binary(caperas_path)
keys=images.keys()

In [4]:
TotalPoints=[]
for i, value in tqdm(enumerate(keys)):
    pointsID=images[value].point3D_ids
    for j in pointsID:
        if(j == -1):
            continue
        else:
            aPoint=[]
            aPoint.append(points3Ds[j].xyz[0])
            aPoint.append(points3Ds[j].xyz[2])
            TotalPoints.append(aPoint)

0it [00:00, ?it/s]

: 

In [ ]:
points = np.array(TotalPoints)
x_min, x_max, y_min, y_max = 700, 1000, -250, 50
x_grid, y_grid = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
kde = gaussian_kde(points.T)
# 그리드 위의 KDE 평가
z_grid = kde(np.vstack([x_grid.ravel(), y_grid.ravel()]))
z_grid = np.reshape(z_grid, x_grid.shape)

# z 값 스케일 조정 (선택적)
scale_factor = 10  # 스케일 인자를 조정하여 z 축의 높이 변경
z_grid_scaled = z_grid * scale_factor

# 3D 시각화
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# 3D 표면 플롯 (조정된 z 값 사용)
surf = ax.plot_surface(x_grid, y_grid, z_grid_scaled, cmap='viridis', edgecolor='none')

# 축 범위 설정
ax.set_xlim(700, 1000)
ax.set_ylim(-250, 50)

# 축 레이블 및 타이틀 설정
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Scaled Density')
ax.set_title('3D Visualization of 2D Coordinates Normal Distribution (Scaled)')

# 컬러바 추가
fig.colorbar(surf, shrink=0.5, aspect=5, label='Scaled Density')

plt.show()

NameError: name 'np' is not defined

In [ ]:
#이거
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


x_min, x_max, y_min, y_max = 700, 1000, -250, 50
x_bins = y_bins = 15  # Number of bins along each axis

# Calculate histogram
hist, xedges, yedges = np.histogram2d(points[:,0], points[:,1], bins=(x_bins, y_bins), range=[[x_min, x_max], [y_min, y_max]])

# Create grid for bar positions
xpos, ypos = np.meshgrid(xedges[:-1] + (xedges[1]-xedges[0])/2, yedges[:-1] + (yedges[1]-yedges[0])/2)
xpos = xpos.flatten('F')
ypos = ypos.flatten('F')
zpos = np.zeros_like(xpos)

# Bar sizes
dx = dy = np.ones_like(zpos) * (xedges[1] - xedges[0])
dz = hist.flatten()

# Plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Normalize the dz values to get the color index
norm = plt.Normalize(dz.min(), dz.max())
#colors = plt.cm.viridis(norm(dz))
colors = plt.cm.cool(norm(dz)) 

# Plot bars with colors
ax.bar3d(xpos, ypos, zpos, dx, dy, dz, color=colors, zsort='average')

# Adding color bar
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap='cool'), shrink=0.5, aspect=5)
cbar.set_label('Frequency')

# Labels & title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Frequency')
ax.set_title('3D Cumulative Distribution Histogram with Color Scale')

plt.show()
